In [28]:
pip install requests

     |████████████████████████████████| 61kB 1.9MB/s eta 0:00:01
     |████████████████████████████████| 153kB 3.1MB/s eta 0:00:01
     |████████████████████████████████| 163kB 8.8MB/s eta 0:00:01
     |████████████████████████████████| 143kB 7.0MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [87]:
import requests
import json

In [88]:
text = open('funkemtxt.txt', 'rb').read().decode(encoding='utf-8')

vocab = sorted(set(text))


In [89]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [90]:
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [91]:
checkpoint_dir ='./training_checkpoints'
tf.train.latest_checkpoint(checkpoint_dir)


'./training_checkpoints/ckpt_50'

In [92]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [93]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            31232     
_________________________________________________________________
lstm_2 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_2 (Dense)              (1, None, 122)            125050    
Total params: 5,403,258
Trainable params: 5,403,258
Non-trainable params: 0
_________________________________________________________________


In [175]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 900

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.0595

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))


In [186]:
funk_generator = generate_text(model, start_string=u"som: ")


In [187]:
print(funk_generator)


som: As menina gosta

Nossa ou não vai tá safadinha
De, de sabia o que é pra não se esquecer
E na praci no seu corpo nu
É se envolver, somos festa em avisando e forma de exemplo
Sou todo lado, na casa dos macho
Pode ter certeza que lá eu me acho
Eu vou dormir lá em baixo, na casa dos macho
Pode ter certeza que lá eu me acho
Eu vou dormir lá em baixo, na casa dos macho
Pode ter certeza que lá eu me acho
Eu vou dormir lá em baixo, na casa dos macho
Pode ter certeza que lá eu me acho
Eu vou dormir lá em baixo, na casa dos macho
Pode ter certeza que lá eu me acho
Eu vou dormir lá em baixo, na casa dos macho
Pode ter certeza que lá eu me acho
Eu vou dormir lá em baixo, na casa dos macho
Pode ter certeza que lá eu me acho
Eu vou dormir lá em baixo, na casa dos macho
Pode ter certeza que lá eu me acho
Eu vou dormir lá em baixo, na casa dos macho
Pode ter certeza que lá eu me acho
Eu vou dormir lá em


In [164]:
API_ENDPOINT = "http://funk-generator.azurewebsites.net/api/funks"


In [165]:
data = {"name": "Bomba e café",
        "author": "MC TensorFlow",
        "letter":funk_generator
    
}

In [166]:
data

{'name': 'Bomba e café',
 'author': 'MC TensorFlow',
 'letter': 'baile: Patráculba, matara no chão\nE quando ela mexe com a bunda no chão\n\nPra isso aqui é (Bomba)\n\nPara dançar isso aqui é (Bomba)\nPara mexer isso aqui é (Bomba)\n\nPara dançar isso aqui\nÉ fora que o meu bonde é pra dar um rolé\n\nHoje não vai ter café\nVocê já sabe qual é\nTática ágil de persuasão\nMete o louco em quem tá no seu pé\nE quando cê percebe, linguinha não é sem só de sacanagem\nVários sapagar minha historia\nVer o mundo tá dentro do cabelo verde\nNa gangue do cabelo verde, na gangue do cabelo verde\nNa gangue do cabelo verde, na gangue do cabelo verde\nNa gangue do cabelo verde, na gangue do cabelo verde, só o pai [?]\nNão é só prazer, por isso tu voltou\nGarota, eu tenho que dizer\nTodas que sentir prazer sua fogo entender a perfeição é de festi\n\nEncarcerados na cama, e sabem gritar!\n\nVou marcar um som\nPra todo lado\nE nssa mistura na cama até merece um som\nPlantentama sem dinheiro e procurando g

In [167]:
headers = {'content-type': 'application/json'}

In [168]:
r = requests.post(API_ENDPOINT, data=json.dumps(data), headers=headers) 



In [169]:
r

<Response [201]>